In [14]:
! pip3 install -r requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [15]:
import glob
import os
import boto3
import socket
from botocore.handlers import disable_signing
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchbnn as bnn

In [17]:
if socket.gethostname() == 'Rohits-MacBook-Pro.local':
    rootdir = '/Users/rohitchanne/Documents/capstone/data/'
else: 
    rootdir = '' # Enter your hone dir here

In [18]:
client = boto3.client('s3', aws_access_key_id='', aws_secret_access_key='')
client._request_signer.sign = (lambda *args, **kwargs: None)

resource = boto3.resource('s3')
resource.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)

### Checking file size within bucket 
*bucket name: cse-cic-ids2018 ref:https://www.unb.ca/cic/datasets/ids-2018.html*

In [19]:
cse_bucket = 'cse-cic-ids2018'
resp = client.list_objects_v2(Bucket=cse_bucket)

In [20]:
for obj in resp['Contents']:
    size = obj["Size"]
    key = obj["Key"]
    if obj["Size"] > 0:
        print(f'key:{key}, size: {round(size/1024/1024,2)} mb')

key:Original Network Traffic and Log data/Friday-02-03-2018/logs.zip, size: 225.77 mb
key:Original Network Traffic and Log data/Friday-02-03-2018/pcap.zip, size: 42714.92 mb
key:Original Network Traffic and Log data/Friday-16-02-2018/logs.zip, size: 148.05 mb
key:Original Network Traffic and Log data/Friday-16-02-2018/pcap.zip, size: 36750.48 mb
key:Original Network Traffic and Log data/Friday-23-02-2018/logs.zip, size: 199.79 mb
key:Original Network Traffic and Log data/Friday-23-02-2018/pcap.zip, size: 56356.36 mb
key:Original Network Traffic and Log data/Thursday-01-03-2018/logs.zip, size: 217.08 mb
key:Original Network Traffic and Log data/Thursday-01-03-2018/pcap.zip, size: 49932.55 mb
key:Original Network Traffic and Log data/Thursday-15-02-2018/logs.zip, size: 142.58 mb
key:Original Network Traffic and Log data/Thursday-15-02-2018/pcap.zip, size: 39370.9 mb
key:Original Network Traffic and Log data/Thursday-22-02-2018/logs.zip, size: 195.28 mb
key:Original Network Traffic and Lo

### Looks like _Processed Traffic Data for ML Algorithms_ folder has the necessary data we need.
####  If you do it on local this will take a while.

In [22]:
for obj in resp['Contents']:
    size = obj["Size"]
    key = obj["Key"]
    file_name = key.split('/')[1]
    if size > 0 and 'Processed Traffic' in key:
        print(f'Downloading key:{key}, size: {round(size/1024/1024,2)} mb')
        resource.meta.client.download_file(cse_bucket, key, rootdir+file_name)
        print(f'{file_name} downloaded')

Friday-02-03-2018_TrafficForML_CICFlowMeter.csv downloaded
Friday-16-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Friday-23-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv downloaded
Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv downloaded


### [Work in progress]Analyzing Data

In [23]:
df_traffic = pd.read_csv(f'{rootdir}Friday-02-03-2018_TrafficForML_CICFlowMeter.csv')

In [24]:
dtypes = df_traffic.dtypes
print(f'Number of features {type(dtypes)}')

Number of features <class 'pandas.core.series.Series'>


In [25]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dtypes)

Dst Port               int64
Protocol               int64
Timestamp             object
Flow Duration          int64
Tot Fwd Pkts           int64
Tot Bwd Pkts           int64
TotLen Fwd Pkts        int64
TotLen Bwd Pkts      float64
Fwd Pkt Len Max        int64
Fwd Pkt Len Min        int64
Fwd Pkt Len Mean     float64
Fwd Pkt Len Std      float64
Bwd Pkt Len Max        int64
Bwd Pkt Len Min        int64
Bwd Pkt Len Mean     float64
Bwd Pkt Len Std      float64
Flow Byts/s          float64
Flow Pkts/s          float64
Flow IAT Mean        float64
Flow IAT Std         float64
Flow IAT Max         float64
Flow IAT Min         float64
Fwd IAT Tot          float64
Fwd IAT Mean         float64
Fwd IAT Std          float64
Fwd IAT Max          float64
Fwd IAT Min          float64
Bwd IAT Tot          float64
Bwd IAT Mean         float64
Bwd IAT Std          float64
Bwd IAT Max          float64
Bwd IAT Min          float64
Fwd PSH Flags          int64
Bwd PSH Flags          int64
Fwd URG Flags 

In [26]:
df_traffic.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,...,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000
mean,8423.42940,8.04985,11514208.72033,5.88728,6.98318,392.08376,5271.27532,210.43115,8.24920,54.61640,...,2.04423,17.45403,122957.85345,61309.72034,190523.05514,83054.45673,4781197.35081,106422.42249,4863777.33665,4677016.56342
std,15161.75671,4.43263,30121132.47167,89.03317,212.95795,2159.77937,308876.83187,266.94967,20.94972,57.62840,...,13.92463,5.17953,2083098.85479,1261694.00443,2761455.52967,1759831.61262,17413075.11403,1462605.49005,17568079.17726,17333032.00493
min,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,80.00000,6.00000,516.00000,2.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,20.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
50%,3389.00000,6.00000,11242.00000,3.00000,1.00000,63.00000,129.00000,46.00000,0.00000,40.00000,...,1.00000,20.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
75%,8080.00000,6.00000,2210150.00000,7.00000,5.00000,365.00000,582.00000,326.00000,0.00000,108.66667,...,3.00000,20.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
max,65534.00000,17.00000,119999997.00000,43159.00000,69241.00000,1100627.00000,101000000.00000,1711.00000,1460.00000,1460.00000,...,9262.00000,44.00000,111000000.00000,74900000.00000,111000000.00000,111000000.00000,120000000.00000,75900000.00000,120000000.00000,120000000.00000


In [27]:
df_traffic.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,02/03/2018 08:47:38,141385,9,7,553,3773.0,202,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,02/03/2018 08:47:38,281,2,1,38,0.0,38,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,02/03/2018 08:47:40,279824,11,15,1086,10527.0,385,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,02/03/2018 08:47:40,132,2,0,0,0.0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,02/03/2018 08:47:41,274016,9,13,1285,6141.0,517,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


### Most of the Data Files have header records within data file so we need to clean them up as they are causing issues in schema detection

In [28]:
dest = rootdir+'data_parquet/'
tables = {}
for obj in resp['Contents']:
    size = obj["Size"]
    key = obj["Key"]
    file_name = key.split('/')[1]
    dest_name = file_name.split('.')[0]+'_clean.parquet'
    
    if size > 0 and 'Processed Traffic' in key:
        print(f'Processing file_name:{file_name}')
        df_temp = pd.read_csv(f'{rootdir}{file_name}')
        #print(df_temp.dtypes)
        df_temp = df_temp[df_temp['Dst Port'] != 'Dst Port'] #Cleaning up necessary headers within rows
        df_temp.to_csv(f'{rootdir}clean_{file_name}',encoding='utf-8', index=False)
        df_temp = pd.read_csv(f'{rootdir}clean_{file_name}')
        tables[file_name] = pa.Table.from_pandas(df_temp)
        pq.write_table(tables[file_name], f'{dest}{dest_name}', compression='snappy')
        print(f'Finished Parquet conversion for file_name:{file_name}')
        

Processing file_name:Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Friday-16-02-2018_TrafficForML_CICFlowMeter.csv


/var/folders/lb/z26czmjn11z59vv2p8rgj_5h0000gn/T/ipykernel_43807/1456713873.py:11: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(f'{rootdir}{file_name}')


Finished Parquet conversion for file_name:Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv


/var/folders/lb/z26czmjn11z59vv2p8rgj_5h0000gn/T/ipykernel_43807/1456713873.py:11: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(f'{rootdir}{file_name}')


Finished Parquet conversion for file_name:Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv


/var/folders/lb/z26czmjn11z59vv2p8rgj_5h0000gn/T/ipykernel_43807/1456713873.py:11: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(f'{rootdir}{file_name}')


Finished Parquet conversion for file_name:Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv


### Checking sizes csv vs parquet:

In [29]:
list_of_files = filter( os.path.isfile, glob.glob(rootdir + '*') )
type(list_of_files)
files_with_size = [ (file_path, os.stat(file_path).st_size) for file_path in list_of_files ]
files_with_size.sort()
for file_path, file_size in files_with_size:
    if 'clean_' in file_path:
        file_name = file_path.split('/')
        print(f'Size_MB: {round(file_size/1024/1024,2)}, Data File: {file_name[-1]}')

Size_MB: 387.13, Data File: clean_Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 340.2, Data File: clean_Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 391.94, Data File: clean_Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 4051.86, Data File: clean_Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 110.12, Data File: clean_Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 385.87, Data File: clean_Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 391.71, Data File: clean_Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 371.47, Data File: clean_Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 344.18, Data File: clean_Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 209.53, Data File: clean_Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv


In [30]:
list_of_files = filter( os.path.isfile, glob.glob(dest + '*') )
type(list_of_files)
files_with_size = [ (file_path, os.stat(file_path).st_size) for file_path in list_of_files ]
files_with_size.sort()
for file_path, file_size in files_with_size:
    if 'parquet' in file_path:
        file_name = file_path.split('/')
        print(f'Size_MB: {round(file_size/1024/1024,2)}, Data File: {file_name[-1]}')

Size_MB: 109.63, Data File: Friday-02-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 92.82, Data File: Friday-16-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 130.97, Data File: Friday-23-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 1099.3, Data File: Thuesday-20-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 36.41, Data File: Thursday-01-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 129.17, Data File: Thursday-15-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 130.86, Data File: Thursday-22-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 97.72, Data File: Wednesday-14-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 61.5, Data File: Wednesday-21-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 70.36, Data File: Wednesday-28-02-2018_TrafficForML_CICFlowMeter_clean.parquet


# Reading data from Parquet

In [31]:
dfs_parquet = {}
for file_path, file_size in files_with_size:
    if 'parquet' in file_path:
        file_name = file_path.split('/')[-1]
        df_name = file_name.split('_')[0]
        print(f'Reading Data File: {file_name}')       
        dfs_parquet[df_name] = pd.read_parquet(file_path, engine='pyarrow')
        

Reading Data File: Friday-02-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-16-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-23-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thuesday-20-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-01-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-15-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-22-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-14-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-21-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-28-02-2018_TrafficForML_CICFlowMeter_clean.parquet


In [32]:
for k,df in dfs_parquet.items():
    print(f'DF:{k}, Shape{df.shape}')
    print(f'Schema:')
    print(df.info())

DF:Friday-02-03-2018, Shape(1048575, 80)
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  float64
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-nu

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  int64  
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-null  float64
 15  Bwd Pkt Len Std    1048575 non-n

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  int64  
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-null  float64
 15  Bwd Pkt Len Std    1048575 non-n

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  int64  
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-null  float64
 15  Bwd Pkt Len Std    1048575 non-n

### Checking Label Values in each data set

In [33]:
for k,df in dfs_parquet.items():
    print(f'DF:{k}, Shape{df.shape}')
    print(df['Label'].unique())

DF:Friday-02-03-2018, Shape(1048575, 80)
['Benign' 'Bot']
DF:Friday-16-02-2018, Shape(1048574, 80)
['Benign' 'DoS attacks-SlowHTTPTest' 'DoS attacks-Hulk']
DF:Friday-23-02-2018, Shape(1048575, 80)
['Benign' 'Brute Force -Web' 'Brute Force -XSS' 'SQL Injection']
DF:Thuesday-20-02-2018, Shape(7948748, 84)
['Benign' 'DDoS attacks-LOIC-HTTP']
DF:Thursday-01-03-2018, Shape(331100, 80)
['Benign' 'Infilteration']
DF:Thursday-15-02-2018, Shape(1048575, 80)
['Benign' 'DoS attacks-GoldenEye' 'DoS attacks-Slowloris']
DF:Thursday-22-02-2018, Shape(1048575, 80)
['Benign' 'Brute Force -Web' 'Brute Force -XSS' 'SQL Injection']
DF:Wednesday-14-02-2018, Shape(1048575, 80)
['Benign' 'FTP-BruteForce' 'SSH-Bruteforce']
DF:Wednesday-21-02-2018, Shape(1048575, 80)
['Benign' 'DDOS attack-LOIC-UDP' 'DDOS attack-HOIC']
DF:Wednesday-28-02-2018, Shape(613071, 80)
['Benign' 'Infilteration']


### Transform data Benign Vs All other values as Allowed vs Not allowed

In [34]:
for k,df in dfs_parquet.items():
    print(k)
    df['is_allowed'] = df['Label'] == 'Benign'
    del df['Label']
    print(f'DF:{k}, Shape{df.shape}')

Friday-02-03-2018
DF:Friday-02-03-2018, Shape(1048575, 80)
Friday-16-02-2018
DF:Friday-16-02-2018, Shape(1048574, 80)
Friday-23-02-2018
DF:Friday-23-02-2018, Shape(1048575, 80)
Thuesday-20-02-2018
DF:Thuesday-20-02-2018, Shape(7948748, 84)
Thursday-01-03-2018
DF:Thursday-01-03-2018, Shape(331100, 80)
Thursday-15-02-2018
DF:Thursday-15-02-2018, Shape(1048575, 80)
Thursday-22-02-2018
DF:Thursday-22-02-2018, Shape(1048575, 80)
Wednesday-14-02-2018
DF:Wednesday-14-02-2018, Shape(1048575, 80)
Wednesday-21-02-2018
DF:Wednesday-21-02-2018, Shape(1048575, 80)
Wednesday-28-02-2018
DF:Wednesday-28-02-2018, Shape(613071, 80)


### Testing out basic bayes

In [35]:
from sklearn.preprocessing import MinMaxScaler
cat_cols = ['Dst Port']

In [36]:
df_train = pd.concat([dfs_parquet['Friday-02-03-2018'], 
                      dfs_parquet['Friday-16-02-2018'], 
                      dfs_parquet['Friday-23-02-2018'],
                      dfs_parquet['Thursday-01-03-2018'],
                      dfs_parquet['Thursday-15-02-2018'],
                      dfs_parquet['Thursday-22-02-2018']
                     ]
                    )
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_train['Timestamp'] = pd.to_datetime(df_train['Timestamp'])
df_train['Date'] = pd.to_datetime(df_train['Timestamp']).dt.date
df_train['TS_relative'] = (df_train['Timestamp'].astype(int) - 
                             pd.to_datetime(df_train['Date']).astype(int))/ 10**9
df_train = df_train.drop(['Timestamp'], axis = 1)
df_train = df_train.drop(['Date'], axis = 1)
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train.fillna(0, inplace=True)
df_train[cat_cols] = df_train[cat_cols].astype('category')
X_train = df_train.drop(['is_allowed'], axis = 1)
y_train = df_train['is_allowed']*1 

In [37]:
stdsc = MinMaxScaler()
stdsc.fit(X_train)
X_train_scaled = stdsc.transform(X_train)

In [39]:
df_test = pd.concat([dfs_parquet['Wednesday-14-02-2018'],
                    dfs_parquet['Wednesday-21-02-2018'],
                    dfs_parquet['Wednesday-28-02-2018']]
                     )
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_test['Timestamp'] = pd.to_datetime(df_test['Timestamp'])
df_test['Date'] = pd.to_datetime(df_test['Timestamp']).dt.date
df_test['TS_relative'] = (df_test['Timestamp'].astype(int) - 
                             pd.to_datetime(df_test['Date']).astype(int))/ 10**9
df_test = df_test.drop(['Timestamp'], axis = 1)
df_test = df_test.drop(['Date'], axis = 1)
df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
df_test.fillna(0, inplace=True)
df_test[cat_cols] = df_test[cat_cols].astype('category')
X_test = df_test.drop(['is_allowed'], axis = 1)
y_test = df_test['is_allowed'].astype(int)*1

In [40]:
stdsc = MinMaxScaler()
stdsc.fit(X_test)
X_test_scaled = stdsc.transform(X_test)

In [66]:
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=79, out_features=128),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=128, out_features=2),
)

ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model.parameters(), lr=0.01)

In [55]:
x_for_nn =  torch.from_numpy(X_train_scaled).float()
x_for_nn.shape

torch.Size([5573974, 79])

In [56]:
y_for_nn = torch.from_numpy(y_train.to_numpy())
y_for_nn.shape

torch.Size([5573974])

In [57]:
x_for_nn_test =  torch.from_numpy(X_test_scaled).float()
x_for_nn_test.shape

torch.Size([2710221, 79])

In [58]:
y_for_nn_test = torch.from_numpy(y_test.to_numpy())
y_for_nn_test.shape

torch.Size([2710221])

In [67]:
kl_weight = 0.1
for step in range(100):
    pre = model(x_for_nn)
    ce = ce_loss(pre, y_for_nn)
    kl = kl_loss(model)
    cost = ce + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if step%10 == 0:
        print(f'Epoch: {step}/100.............')

    
_, predicted = torch.max(pre.data, 1)
print(f'F1: {f1_score(y_for_nn, predicted)}')
print(f'Accuracy: {accuracy_score(y_for_nn, predicted)}')
print(f'Precision: {precision_score(y_for_nn, predicted)}')
print(f'Recall: {recall_score(y_for_nn, predicted)}')

Epoch: 0/100.............
Epoch: 10/100.............
Epoch: 20/100.............
Epoch: 30/100.............
Epoch: 40/100.............
Epoch: 50/100.............
Epoch: 60/100.............
Epoch: 70/100.............
Epoch: 80/100.............
Epoch: 90/100.............
F1: 0.9486675270777175
Accuracy: 0.9164136754136277
Precision: 0.9489500285444114
Recall: 0.9483851937617689


In [72]:
pre_test = model(x_for_nn_test)
_, predicted_test = torch.max(pre_test.data, 1)

In [73]:
print(f'F1: {f1_score(y_for_nn_test, predicted_test)}')
print(f'Accuracy: {accuracy_score(y_for_nn_test, predicted_test)}')
print(f'Precision: {precision_score(y_for_nn_test, predicted_test)}')
print(f'Recall: {recall_score(y_for_nn_test, predicted_test)}')

F1: 0.9337487641242374
Accuracy: 0.9233793111336677
Precision: 0.9370075261340105
Recall: 0.9305125904598518


### RNN model

In [74]:
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, Masking, Embedding

In [75]:
df_train_rnn = pd.concat([dfs_parquet['Friday-02-03-2018'], 
                      dfs_parquet['Thursday-22-02-2018']
                     ]
                    )
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_train_rnn['Timestamp'] = pd.to_datetime(df_train_rnn['Timestamp'])
df_train_rnn['Date'] = pd.to_datetime(df_train_rnn['Timestamp']).dt.date
df_train_rnn['TS_relative'] = (df_train_rnn['Timestamp'].astype(int) - 
                             pd.to_datetime(df_train_rnn['Date']).astype(int))/ 10**9
df_train_rnn = df_train_rnn.drop(['Timestamp'], axis = 1)
df_train_rnn = df_train_rnn.drop(['Date'], axis = 1)
df_train_rnn.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train_rnn.fillna(0, inplace=True)
df_train_rnn[cat_cols] = df_train_rnn[cat_cols].astype('category')
X_train = df_train_rnn.drop(['is_allowed'], axis = 1)
y_train = df_train_rnn['is_allowed']*1 

In [76]:
stdsc = MinMaxScaler()
stdsc.fit(X_train)
X_train_scaled = stdsc.transform(X_train)

In [50]:
model_rnn = Sequential()

2022-06-25 07:04:36.691417: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [51]:
model_rnn.add(LSTM(units = 128, return_sequences = True, input_shape = (X_train_scaled.shape[1], 1)))
model_rnn.add(Dropout(0.1))
model_rnn.add(LSTM(units = 128))
model_rnn.add(Dropout(0.1))

In [52]:
model_rnn.compile(optimizer = 'adam', loss = 'binary_crossentropy')

In [53]:
#model_rnn.fit(X_train_scaled, y_train, epochs = 10, batch_size = 100000)